In [ ]:
# Get Data
from general_functions.file_operations import read_txt_file
data_folder = "../data/"
text_file_location = data_folder + "extracted_chapters/Chapter_1.txt"
text = read_txt_file(text_file_location)
print(text)

Run BookNLP pipeline, create 

In [ ]:
## BookNLP Stuff
from general_functions.booknlp import run_booknlp

# Constants 
# Input file to process
input_file="../data/dahl-boy-4.txt"
# Output directory to store resulting files in 
output="../data/booknlp_output/boy/"
# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
id="boy"
pipeline="entity,quote,supersense,event,coref"

#Call functions
run_booknlp(input_file_location=input_file, output_directory=output, book_id=id,pipeline=pipeline)


In [ ]:
#Suspense chart 
import pandas as pd
import spacy
import json
import matplotlib.pyplot as plt

# Constants
lexicon_path = "../data/NRC-VAD-Lexicon.csv"
#Always use a relative path inside the project directory
story_path = "../data/dahl-boy-4.txt"
output_path = "../data/suspense.json"

# Load the English model for SpaCy
nlp = spacy.load("en_core_web_sm")

def load_lexicon(file_path):
    return pd.read_csv(file_path, sep="\t")

def read_and_split_story(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return [para.strip() for para in text.split("\n\n") if para]

def get_negative_valence_arousal_score(sentence, lexicon_df):
    doc = nlp(sentence)
    scores = []
    for token in doc:
        word = token.text
        if not word.isalnum():
            continue
        word_data = lexicon_df.loc[lexicon_df['Word'] == word]
        if not word_data.empty:
            valence = word_data['Valence'].values[0]
            arousal = word_data['Arousal'].values[0]
            negative_valence = 1 - valence
            scores.append(negative_valence + arousal)
    if scores:
        return sum(scores) / len(scores)
    else:
        return None

def analyze_paragraphs(paragraphs, lexicon_df):
    para_scores = {}
    for index, para in enumerate(paragraphs):
        sentence_scores = []
        doc = nlp(para)
        max_sent = ""
        max_score = -100
        for sent in doc.sents:
            score = get_negative_valence_arousal_score(sent.text, lexicon_df)
            if score and score > max_score:
                max_score = score
                max_sent = sent.text
            sentence_scores.append(score)
        sentence_scores = [score for score in sentence_scores if score is not None]
        if not sentence_scores:
            print(para)
            continue
        average_score = sum(sentence_scores) / len(sentence_scores)
        para_scores[index + 1] = [average_score, max_sent]
    return para_scores

def write_to_json(data, output_path):
    with open(output_path, "w") as json_file:
        json.dump(data, json_file)

# Load lexicon data
lexicon_df = load_lexicon(lexicon_path)

# Extract paragraphs from the story
paragraphs = read_and_split_story(story_path)

# Analyze the sentiment of paragraphs
results = analyze_paragraphs(paragraphs, lexicon_df)

# Save results to a JSON file
write_to_json(results, output_path)
